In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [2]:
from langmem import create_memory_manager
from pydantic import BaseModel
from langchain_openai.chat_models import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

class Person(BaseModel):
    """Store a person's name, role, and preferences."""
    name: str
    role: str
    preferences: list[str] | None = None


manager = create_memory_manager(
    llm,
    schemas=[Person],
    instructions="Extract people's names, roles, and any mentioned preferences.",
    enable_inserts=True,
    enable_updates=True,
    enable_deletes=True,
)

In [3]:
conversation = [
    {
        "role": "user",
        "content": (
            "John is a senior developer who loves coffee. "
            "Alice is a junior developer who hates coffee."
        )
    }
]
memories = manager.invoke({"messages": conversation})
print(memories)

[ExtractedMemory(id='7fcaa043-046d-4b25-9af9-23741ddc4134', content=Person(name='John', role='senior developer', preferences=['coffee'])), ExtractedMemory(id='7d065021-b8e1-4352-9a74-06fa8cc4f764', content=Person(name='Alice', role='junior developer', preferences=['hates coffee']))]


In [4]:
try:
    conversation_no_extraction = [
        {
            "role": "user",
            "content": (
                "Today it rained for two hours, and then the sun came out."
            )
        }
    ]

    memories_no_extraction = manager.invoke({"messages": conversation_no_extraction})
    print(memories_no_extraction)
except Exception as e:
    print(e)

Error code: 500 - {'error': {'message': 'The model produced invalid content. Consider modifying your prompt if you are seeing this error persistently.', 'type': 'model_error', 'param': None, 'code': None}}


In [5]:
from langmem import create_memory_store_manager

namespace=("memories",)

memory_manager = create_memory_store_manager(
    llm,
    namespace=namespace,
    instructions="Only save information related about food the user likes"
)
try:
    memory_manager.invoke({"messages": ["I like dogs. My dog's name is Fido."]})
except Exception as e:
    print("Exception:", e)

Exception: Called get_config outside of a runnable context


In [10]:
from langmem import ReflectionExecutor

executor = ReflectionExecutor(memory_manager)

In [11]:
from langgraph.store.memory import InMemoryStore
from langgraph.func import entrypoint

store = InMemoryStore(
    index={
        "dims": 1536,
        "embed": "openai:text-embedding-3-small",
    }
)

@entrypoint(store=store)
async def chat(message: str):
    response = llm.invoke(message)

    to_process = {"messages": [{"role": "user", "content": message}] + [response]}
    # await memory_manager.ainvoke(to_process)
    executor.submit(to_process, after_seconds=1)
    return response.content

In [12]:
response = await chat.ainvoke(
    "I like to eat Pizza",
)

In [13]:
try:
    response = await chat.ainvoke(
        "I like dogs. My dog's name is Fido.",
    )
except Exception as e:
    print("Exception:", e)

In [14]:
store.search(namespace)

[Item(namespace=['memories'], key='670da84c-c048-44fe-8bde-c4f5377ae95e', value={'kind': 'Memory', 'content': {'content': 'User likes pizza.'}}, created_at='2025-03-08T16:12:08.805877+00:00', updated_at='2025-03-08T16:12:08.805877+00:00', score=None)]

### Better approach - Tools!

In [16]:
from langgraph.store.memory import InMemoryStore
from langgraph.prebuilt import create_react_agent
from langmem import create_manage_memory_tool, create_search_memory_tool


store = InMemoryStore(
    index={
        "dims": 1536,
        "embed": "openai:text-embedding-3-small",
    }
)


In [17]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

tools=[
    create_manage_memory_tool(namespace=("memories", "{user_id}"), store=store),
    create_search_memory_tool(namespace=("memories", "{user_id}"), store=store),
]

In [18]:
app = create_react_agent(llm, tools=tools)

In [19]:
app.invoke({"messages": [{"role": "user", "content": "hi!"}]}, config={"configurable": {"user_id": "alice"}})

{'messages': [HumanMessage(content='hi!', additional_kwargs={}, response_metadata={}, id='6fb5b4bc-f670-43e4-8975-df9c33635004'),
  AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 241, 'total_tokens': 252, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737a9306', 'finish_reason': 'stop', 'logprobs': None}, id='run-e7321c74-1828-4ce2-b22a-23f361fb597f-0', usage_metadata={'input_tokens': 241, 'output_tokens': 11, 'total_tokens': 252, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}

In [20]:
app.invoke({"messages": [{"role": "user", "content": "what do you know about me?"}]}, config={"configurable": {"user_id": "alice"}})

{'messages': [HumanMessage(content='what do you know about me?', additional_kwargs={}, response_metadata={}, id='0207f037-b34c-429b-975e-82ae56aadbaa'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_HLqHXNhU3j7JeL8PNYK50GgZ', 'function': {'arguments': '{"query":"user"}', 'name': 'search_memory'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 246, 'total_tokens': 261, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737a9306', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-2b1b703b-5ff1-4996-ac3b-0781af78b824-0', tool_calls=[{'name': 'search_memory', 'args': {'query': 'user'}, 'id': 'call_HLqHXNhU3j7JeL8PNYK50GgZ', 'type': 'tool_call'}], usage_metadata={'input_

In [21]:
app.invoke({"messages": [{"role": "user", "content": "I love spaghetti"}]}, config={"configurable": {"user_id": "alice"}})

{'messages': [HumanMessage(content='I love spaghetti', additional_kwargs={}, response_metadata={}, id='9ac48e9d-5624-46bf-9a42-8bb44718403a'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_IOMqnQdUMBahmicU2xYBC0cz', 'function': {'arguments': '{"content":"User loves spaghetti","action":"create"}', 'name': 'manage_memory'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 242, 'total_tokens': 263, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737a9306', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-f8126c89-28a0-4973-9261-81368a326eed-0', tool_calls=[{'name': 'manage_memory', 'args': {'content': 'User loves spaghetti', 'action': 'create'}, 'id': 'call_IOMqnQdUM

In [22]:
app.invoke({"messages": [{"role": "user", "content": "what do you know about me?"}]}, config={"configurable": {"user_id": "alice"}})

{'messages': [HumanMessage(content='what do you know about me?', additional_kwargs={}, response_metadata={}, id='018b5b16-312e-480c-81d9-13e0418c0565'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_uvGMWOgqZ8wxlzrcw8MyU9sj', 'function': {'arguments': '{"query":"user"}', 'name': 'search_memory'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 246, 'total_tokens': 261, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737a9306', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-d402dccc-26b2-43d4-acf7-1060d983dc80-0', tool_calls=[{'name': 'search_memory', 'args': {'query': 'user'}, 'id': 'call_uvGMWOgqZ8wxlzrcw8MyU9sj', 'type': 'tool_call'}], usage_metadata={'input_

In [23]:
app.invoke({"messages": [{"role": "user", "content": "what do you know about me?"}]}, config={"configurable": {"user_id": "max"}})

{'messages': [HumanMessage(content='what do you know about me?', additional_kwargs={}, response_metadata={}, id='41727410-c9ab-45a1-a603-7b380112fe0a'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_qyoIPTtyhE0PWkXB8RyxC1RD', 'function': {'arguments': '{"query":"user","limit":5}', 'name': 'search_memory'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 246, 'total_tokens': 265, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737a9306', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-fd1d3838-37ea-4751-b5e8-0c5b93d017e5-0', tool_calls=[{'name': 'search_memory', 'args': {'query': 'user', 'limit': 5}, 'id': 'call_qyoIPTtyhE0PWkXB8RyxC1RD', 'type': 'tool_call'}], u

### Prededual Memory: System Instructions

In [24]:
from langmem import create_prompt_optimizer

optimizer = create_prompt_optimizer(
    llm,
    kind="metaprompt",
    config={"max_reflection_steps": 3}
)

In [26]:
prompt = "You are a helpful assistant."
trajectory = [
    {"role": "user", "content": "Explain inheritance in Python"},
    {"role": "assistant", "content": "Here's a detailed theoretical explanation..."},
    {"role": "user", "content": "Show me a practical example instead"},
]
optimized = optimizer.invoke({
    "trajectories": [(trajectory, {"user_score": 0})],
    "prompt": prompt
})
print(optimized)

You are a helpful assistant. When users ask for explanations, especially in programming contexts, prioritize providing practical examples alongside theoretical explanations.
